## Step 1:
Let the program know where to find your tools file.

In [2]:
import sys, os
sys.path.append(os.path.join(os.path.realpath(".."), "Tools"))

## Step 2:
Load up all of the necessary packages

In [4]:
# Loading up required packages
from plastid import BAMGenomeArray, VariableFivePrimeMapFactory, \
                        GTF2_TranscriptAssembler, GFF3_TranscriptAssembler, \
                        Transcript, ThreePrimeMapFactory, CenterMapFactory
from plastid.plotting.plots import *
import numpy as np
import os
from Bio import SeqIO
import numpy
import math
import pandas as pd
from scipy import stats
import utilities as utils
from statsmodels.nonparametric.smoothers_lowess import lowess
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from scipy.sparse.linalg import lsqr

## Step 3:
Define the paths to our reference genome and annotation files as well as the path to our data for the Autism Spectrum Disorder (ASD) genes identified by Sfari, the top 200 genes whose regulation is altered by FMRP as determined by RiboDiff, The list of genes and their associated TPM values (used to select for genes with sufficient TPM), and the list of primary transcript isoforms provided by APRIS.

In [114]:
reference_path = "/home/keeganfl/Desktop/Work_Fall_2021/Protocol_test/genome/dmel/"
data_path = "/home/keeganfl/Desktop/measuring_binding_domains/data/"
save_path = "/home/keeganfl/Desktop/measuring_binding_domains/Extra_tables/TE_tables/"

## Step 4: 
load the transcript files so that we can find the sequence of each gene

In [8]:
# Loading up the transcrip[t information.
transcripts = list(GTF2_TranscriptAssembler(open(reference_path + "Drosophila_melanogaster.BDGP6.32.103.gtf"),return_type=Transcript))

## Step 5:
Create a table that lists all genes and their associated lengths. 

### Right now I am using cds length instead of full lenght, I think that is correct. 

In [32]:
lengths_cds = []
gene_name = []
gene_id = []
for tran in transcripts:
    try:
        tran.attr["gene_name"]
    except:
        continue
    try:
        lengths_cds.append(tran.cds_end - tran.cds_start)
    except:
        continue
    gene_name.append(tran.attr["gene_name"])
    gene_id.append(tran.attr["gene_id"])
all_genes = pd.DataFrame(list(zip(gene_name, gene_id, lengths_cds)))
all_genes.columns = ["gene_name", "gene_id", "lengths_cds"]

## Step 6:
Load up the list of APRIS genes. This list will include the transcript IDs of all of the primary isoforms, and will be used to select a single transcript for all genes.

In [44]:
# Load up the Apris genes. 
prin_trans = pd.read_csv(data_path + "apris_fly_data.txt", names = ["gene_name", "???", "gene_id", "rank"])
prin_trans.drop(columns = ["???", "gene_id", "rank"], inplace = True)

## Step 7:
Merge the list of APRIS genes with the list of all gene lengths in order to remove the non-primary isoforms. 

In [47]:
# Merge the count table with the table of APRIS genes
prime_genes = pd.merge(all_genes, prin_trans, how="left", on="gene_name", indicator = True)

# Only keep counts with a transcript ID that matched one of the APRIS primary IDs.
prime_genes = prime_genes.loc[prime_genes._merge == "both"].copy()

# Drop any duplicate genes (duplicates may remain if multiple transcripts have identical cds regions).
prime_genes.drop_duplicates(subset ="gene_name",keep = "first", inplace = True) 

# Remove unnecesary columns. 
prime_genes.drop(columns = ["_merge"], inplace = True)

## Step 8:
Load up the list counts from each example and then merge them with the list of the pimary gene lengths.

In [133]:
# Load up count data.
fly_counts = pd.read_csv(data_path + "counts/fly_counts.csv")

# Merge with gene length data. 
len_counts = pd.merge(prime_genes, fly_counts, how = "inner", left_on = "gene_id", right_on = "Entry")

## Step 9: 
Calculate TPM for each column using the equation 
<br />
$TPM = 10^6 * \frac{reads\: mapped\: to\: the\: transcript\:/\: transcript\: length }{Sum(reads\: mapped\: to\: each\: transcript\:/\: each\: transcript's\: length)}
$
<br />
Then add them as new columns to the pandas dataframe.

In [134]:
# Calculate RPK for each gene so that we can use it to calculate TPM. 
for i in len_counts.columns[4:24]:
    rpk = len_counts[i]/len_counts["lengths_cds"]
    len_counts["%s_rpk" % (i)] = rpk

In [135]:
# Calculate TPM for each gene. 
for i in len_counts.columns[4:24]:
    per_m_s = sum(len_counts["%s_rpk" % i])/1e6
    len_counts["%s_tpm" % (i)] = (len_counts["%s_rpk" % i])/per_m_s

In [136]:
# Drop all of the RPK columns as we only needed them to get TPM. 
len_counts.drop(columns = ['RbCtlR1_rpk', 'RbCtlR2_rpk', 'RbCtlR3_rpk', 'RbCtlR4_rpk',
       'RbTrtR1_rpk', 'RbTrtR2_rpk', 'RbTrtR3_rpk', 'RbTrtR4_rpk',
       'RbTrtR5_rpk', 'RbTrtR6_rpk', 'RbTrtR7_rpk', 'RnaCtlR1_rpk',
       'RnaCtlR2_rpk', 'RnaCtlR3_rpk', 'RnaTrtR1_rpk', 'RnaTrtR2_rpk',
       'RnaTrtR3_rpk', 'RnaTrtR4_rpk', 'RnaTrtR5_rpk', 'RnaTrtR6_rpk'], inplace = True)

## Step 10:
Calculate the Average TPM for the mutant and control RNA and RB samples and then calculate average TE. 

In [137]:
# Avergae tpm for the ribosome profiling controls
len_counts['RbCtl_avg_tpm'] = len_counts[['RbCtlR1_tpm', 'RbCtlR2_tpm', 'RbCtlR3_tpm', 'RbCtlR4_tpm']].mean(axis = 1)

# Average tpm for ribosome profiling treated
len_counts['RbTrt_avg_tpm'] = len_counts[['RbTrtR1_tpm', 'RbTrtR2_tpm', 'RbTrtR3_tpm', 'RbTrtR4_tpm',
            'RbTrtR5_tpm', 'RbTrtR6_tpm', 'RbTrtR7_tpm']].mean(axis = 1)

# Average tpm for the RNA sequncing control
len_counts['RnaCtl_avg_tpm'] = len_counts[['RnaCtlR1_tpm', 'RnaCtlR2_tpm', 'RnaCtlR3_tpm']].mean(axis = 1)

# Avergae tpm for the RNA sequencing treated
len_counts['RnaTrt_avg_tpm'] = len_counts[['RnaTrtR1_tpm', 'RnaTrtR2_tpm','RnaTrtR3_tpm',
            'RnaTrtR4_tpm', 'RnaTrtR5_tpm', 'RnaTrtR6_tpm']].mean(axis= 1)

In [138]:
len_counts['Ctl_TE'] = len_counts['RbCtl_avg_tpm']/len_counts['RnaCtl_avg_tpm']
len_counts['Trt_TE'] = len_counts['RbTrt_avg_tpm']/len_counts['RnaTrt_avg_tpm']
len_counts['FldChng_TE'] = len_counts['Trt_TE']/len_counts['Ctl_TE']

## Step 11
Drop uneccessary columns and save the csv. 

In [139]:
len_counts.drop(columns = ["gene_id", "lengths_cds"], inplace = True)
len_counts.to_csv(save_path + "TE_fly.csv", index = False)

In [140]:
len_counts

,gene_name,Entry,RbCtlR1,RbCtlR2,RbCtlR3,RbCtlR4,RbTrtR1,RbTrtR2,RbTrtR3,RbTrtR4,...,RnaTrtR4_tpm,RnaTrtR5_tpm,RnaTrtR6_tpm,RbCtl_avg_tpm,RbTrt_avg_tpm,RnaCtl_avg_tpm,RnaTrt_avg_tpm,Ctl_TE,Trt_TE,FldChng_TE
0,l(2)gl,FBgn0002121,237,441,526,229,551,357,307,189,...,154.223986,136.910824,156.239688,53.425168,56.315812,140.098918,130.886259,0.381339,0.430265,1.128302
1,Ir21a,FBgn0031209,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.021651,0.000000,0.000000,0.003740,0.013918,0.000000,0.000000,NaN
2,Cda5,FBgn0051973,8,12,32,6,6,14,22,14,...,0.607419,0.580921,0.583157,2.674754,2.791984,0.626519,0.627173,4.269234,4.451699,1.042740
3,dbr,FBgn0067779,63,91,89,39,94,44,43,49,...,23.171139,22.104968,22.253418,12.191389,11.234609,28.237695,22.339551,0.431742,0.502902,1.164822
4,galectin,FBgn0031213,138,145,176,112,226,116,158,98,...,90.904270,96.589732,87.513229,64.914043,67.697021,99.789370,90.585432,0.650511,0.747328,1.148833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13916,mt:ND4,FBgn0262952,1357,761,1005,686,1467,1245,1749,986,...,296.012285,347.866626,288.702637,401.562408,656.833801,249.855675,295.971960,1.607177,2.219243,1.380833
13917,mt:ND4L,FBgn0013683,565,278,414,379,725,537,743,501,...,8.163787,10.328018,7.902835,818.041533,1393.891656,11.438269,8.284705,71.517947,168.248807,2.352540
13918,mt:ND6,FBgn0013685,530,298,462,402,873,706,961,351,...,3.810586,5.280517,6.205607,465.734996,834.842860,4.770220,5.724264,97.633869,145.842820,1.493773
13919,mt:Cyt-b,FBgn0013678,2907,2990,3141,2346,6194,3874,5443,1715,...,2534.633300,2789.589683,2305.889251,1388.632526,2209.783924,2410.018831,2511.132481,0.576192,0.879995,1.527261
